# 1. Workspace preparation

In [ ]:
!pip install pygsheets

In [ ]:
import pandas as pd
import pygsheets
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Connect with database

In [ ]:
# Production database

# conn = psycopg2.connect(
#         host='ec2-3-230-199-240.compute-1.amazonaws.com',
#         database='d3df54jud662it',
#         user='avcobtmtntvzvv',
#         password='9f675a820ce2a07d446dd03981071563fedb390ed9ccbbacf8cc3b70e7d16e0e')

In [ ]:
# Test database

conn = psycopg2.connect(
        host='ec2-34-204-127-36.compute-1.amazonaws.com',
        database='d835d7a38dbqc3',
        user='fzlgrdnpjdjbsq',
        password='1e36f451eed5251a001fe5d674909de72d22b9098efbbbd912cc9c2f19690d21')

# 3. Create function to get data in dataframes

Queries

In [ ]:
sessions = '''SELECT *, EXTRACT (MONTH FROM date_login) AS active_month FROM sessions;'''
users = '''SELECT *, EXTRACT (MONTH FROM created_at) AS created_month FROM users'''
professionals = '''SELECT * FROM professionals'''
countries = '''SELECT * FROM countries'''
countries_cities = '''SELECT * FROM countries_cities ORDER BY country_id'''
cities = '''SELECT * FROM cities ORDER BY City_id'''

function to create dataframes

In [ ]:
def create_df(query):
  cursor_table = conn.cursor()
  cursor_table.execute(query)
  records = cursor_table.fetchall()
  col_names = []
  for col_name in cursor_table.description:
    col_names.append(col_name[0])
  df = pd.DataFrame(records, columns=col_names)
  return df

# 4. Create dataframes

## Sessions

In [ ]:
df_sessions = create_df(sessions)
df_sessions

,session_id,ip_address,devices,OS,browser,date_login,date_logout,user_id,active_month
0,1502,116.122.16.192,smartphone,"""OS"" X",Opera,2021-03-16 18:55:14,2021-03-16 19:37:40,573,3.0
1,1501,22.163.32.101,desktop,"Chrome ""OS""",Chrome,2021-06-05 04:39:48,2021-06-05 06:39:54,542,6.0
2,1503,119.228.238.82,TV,Unknown,Chrome,2021-05-07 12:17:19,2021-05-07 13:27:47,642,5.0
3,1500,249.89.233.115,smartphone,Others,Firefox,2021-02-16 19:11:53,2021-02-16 20:07:23,565,2.0
4,1504,222.87.242.29,desktop,"""OS"" X",Firefox,2021-09-15 23:02:58,2021-09-15 23:34:25,629,9.0
...,...,...,...,...,...,...,...,...,...
995,2462,211.81.150.123,smartphone,Others,Unknown,2021-12-15 17:52:49,2021-12-15 20:15:17,68,12.0
996,2471,87.73.63.103,smartphone,Others,Unknown,2021-11-29 17:07:18,2021-11-29 17:26:11,520,11.0
997,2480,127.145.48.73,smartphone,Others,Android,2021-06-23 22:44:39,2021-06-24 00:55:11,636,6.0
998,2490,209.56.38.146,desktop,"Chrome ""OS""",Chrome,2021-09-30 18:20:34,2021-09-30 20:08:27,608,9.0


## Users

In [ ]:
df_users = create_df(users)
df_users

,user_id,name,email,password,profile_picture,active,created_at,created_by,updated_at,updated_by,role_id,created_month
0,1,hfranssen0,efrisch0@is.gd,Qnl2iIl,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",True,2021-03-30 22:21:16,1,2021-06-30 22:21:16,1,1,3.0
1,2,cbiskupiak1,bgallally1@hibu.com,sgWXz3oU,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",True,2021-01-24 16:02:18,2,2021-05-24 16:02:18,2,2,1.0
2,3,wshipley2,rogiany2@xing.com,zyyNAJKdv,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",True,2021-09-17 14:19:43,3,2021-11-17 14:19:43,3,1,9.0
3,4,fgreenroad3,aison3@sogou.com,7uPXCEiCHk,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-06-02 20:18:53,4,2021-07-02 20:18:53,4,2,6.0
4,5,bklimuk4,wbaudin4@tuttocitta.it,u2CXwnn1GRM,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-05-23 15:41:51,5,2021-07-23 15:41:51,5,1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,achalkrn,fbalkwillrn@sohu.com,haNJCyG0Q,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-05-07 06:44:39,996,2021-06-07 06:44:39,996,2,5.0
996,997,mfouracrero,ccastanaresro@google.es,4sECqcyJ,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-08-01 09:51:25,997,2021-12-01 09:51:25,997,1,8.0
997,998,amottramrp,noughtrightrp@smh.com.au,haCzNsBkr4T,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-02-05 02:33:19,998,2021-06-05 02:33:19,998,2,2.0
998,999,rmccroryrq,rcantyrq@loc.gov,xFafs4t,"[b'h', b't', b't', b'p', b's', b':', b'/', b'/...",False,2021-06-29 03:54:49,999,2021-07-29 03:54:49,999,1,6.0


In [ ]:
df_users.rename(columns={'created_at': 'user_created_at'}, inplace=True)
df_users.drop(['name','email','password','profile_picture','created_by','updated_at','updated_by'], 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


## Professionals

In [ ]:
df_professionals = create_df(professionals)
df_professionals.drop(['first_name','last_name','birth_date','phone_number','subdomain','description','created_at','created_by','updated_at','updated_by'], 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
df_professionals

,professional_id,gender,english_level,looking_job,location_preference,location,user_id
0,2,Female to Male,B1,True,Costa Rica,10,2
1,3,FTM,B2,False,Paraguay,7,3
2,6,MTF,A2,True,Costa Rica,4,6
3,7,Non-binary,C1,True,Honduras,15,7
4,10,Gender Questioning,C2,False,Argentina,13,10
...,...,...,...,...,...,...,...
995,932,Male to Female,B2,True,Honduras,15,932
996,945,Intersex,B1,False,Brazil,2,945
997,972,Gender Questioning,C1,False,Chile,13,972
998,993,Trans Person,C1,False,Peru,3,993


## Countries

In [ ]:
df_countries = create_df(countries)
df_countries

,country_id,name,created_at,created_by,updated_at,updated_by
0,1,Argentina,2021-09-29 21:57:17,1,2021-12-22 07:23:12,1
1,2,Bolivia,2021-08-06 02:26:30,2,2022-01-02 10:45:10,2
2,3,Brazil,2021-08-13 00:13:02,3,2021-10-26 16:18:02,3
3,4,Chile,2021-08-28 18:31:40,4,2021-10-10 06:35:47,4
4,5,Colombia,2021-07-22 19:13:48,5,2021-09-14 08:17:05,5
5,6,Costa Rica,2021-01-18 00:43:00,6,2021-03-18 23:30:20,6
6,7,Ecuador,2021-08-02 16:09:41,7,2021-11-20 21:57:38,7
7,8,Guatemala,2021-02-26 21:06:04,8,2021-03-28 01:25:22,8
8,9,Honduras,2021-01-18 06:16:42,9,2021-03-19 23:42:28,9
9,10,Mexico,2021-05-22 21:41:33,10,2021-10-06 12:34:42,10


In [ ]:
df_countries.rename(columns={'name': 'country_name'}, inplace=True)
df_countries.drop(['created_at','created_by','updated_at','updated_by'], 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


## Countries_cities

In [ ]:
df_countries_cities = create_df(countries_cities)
df_countries_cities

,country_id,city_id
0,1,950
1,1,329
2,1,393
3,1,392
4,1,383
...,...,...
995,13,168
996,13,931
997,14,59
998,15,416


## Cities

In [ ]:
df_cities = create_df(cities)
df_cities

,city_id,name,created_at,created_by,updated_at,updated_by
0,1,City of Goiânia,2021-05-31 14:04:20,1,2021-09-16 07:18:24,1
1,2,Guarco,2021-03-19 22:30:55,2,2021-07-10 04:09:48,2
2,3,Alcaldía Distrital de Santa Marta,2021-08-01 23:43:08,3,2021-10-22 22:57:04,3
3,4,Corrales,2021-06-28 09:42:22,4,2021-10-04 22:32:03,4
4,5,City of Buenos Aires,2021-07-06 00:14:03,5,2021-09-14 11:42:53,5
...,...,...,...,...,...,...
995,996,JIRN Estado de Jalisco (JINOR),2021-10-29 01:27:11,996,2021-12-08 08:53:19,996
996,997,Ayuntamiento de Victoria (Tamaulipas),2021-04-27 10:11:03,997,2021-07-14 17:06:50,997
997,998,Arteaga,2021-04-11 16:38:06,998,2021-06-29 05:53:01,998
998,999,San Sebastián de Mariquita,2021-10-06 00:50:44,999,2022-02-15 13:25:31,999


In [ ]:
df_cities.rename(columns={'name': 'city_name'}, inplace=True)
df_cities.drop(['created_at','created_by','updated_at','updated_by'], 1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


# 5. Merge all dataframes

## Cities and Countries

In [ ]:
df_countries_pivote = pd.merge(df_countries, df_countries_cities, on='country_id')

In [ ]:
df_cities_and_countries = pd.merge(df_countries_pivote, df_cities, on='city_id')

## Professionals and users

In [ ]:
df_professionals_users = pd.merge(df_professionals, df_users, on='user_id')
df_professionals_users

,professional_id,gender,english_level,looking_job,location_preference,location,user_id,active,user_created_at,role_id,created_month
0,2,Female to Male,B1,True,Costa Rica,10,2,True,2021-01-24 16:02:18,2,1.0
1,3,FTM,B2,False,Paraguay,7,3,True,2021-09-17 14:19:43,1,9.0
2,6,MTF,A2,True,Costa Rica,4,6,True,2021-09-23 22:28:28,2,9.0
3,7,Non-binary,C1,True,Honduras,15,7,True,2021-05-03 16:59:01,1,5.0
4,10,Gender Questioning,C2,False,Argentina,13,10,False,2021-05-17 02:47:58,2,5.0
...,...,...,...,...,...,...,...,...,...,...,...
995,932,Male to Female,B2,True,Honduras,15,932,True,2021-01-29 09:01:52,2,1.0
996,945,Intersex,B1,False,Brazil,2,945,False,2021-04-09 01:47:59,1,4.0
997,972,Gender Questioning,C1,False,Chile,13,972,False,2021-03-21 18:02:43,2,3.0
998,993,Trans Person,C1,False,Peru,3,993,True,2021-03-24 14:28:01,1,3.0


## Professional_users and countries

In [ ]:
df_professionals_countries = pd.merge(df_professionals_users, df_countries, left_on='location', right_on='country_id')
df_professionals_countries

,professional_id,gender,english_level,looking_job,location_preference,location,user_id,active,user_created_at,role_id,created_month,country_id,country_name
0,2,Female to Male,B1,True,Costa Rica,10,2,True,2021-01-24 16:02:18,2,1.0,10,Mexico
1,60,Cisgender Female,A1,False,Peru,10,60,False,2021-08-23 01:38:59,2,8.0,10,Mexico
2,179,Trans Man,A1,False,Guatemala,10,179,False,2021-07-30 12:24:40,1,7.0,10,Mexico
3,4,Gender Nonconforming,C1,True,Costa Rica,10,4,False,2021-06-02 20:18:53,2,6.0,10,Mexico
4,111,Transgender Woman,C2,True,Venezuela,10,111,False,2021-06-23 01:48:15,1,6.0,10,Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,349,Female to Male,C2,True,Mexico,12,349,True,2021-07-20 03:44:12,1,7.0,12,Paraguay
996,510,Transsexual Person,A2,True,Bolivia,12,510,True,2021-02-15 19:50:11,2,2.0,12,Paraguay
997,605,Cisgender,A2,True,Costa Rica,12,605,False,2021-02-10 00:55:57,1,2.0,12,Paraguay
998,705,Transfeminine,A1,True,Venezuela,12,705,True,2021-06-04 14:05:55,1,6.0,12,Paraguay


## Merged dataframes

In [ ]:
df_merged = pd.merge(df_sessions, df_professionals_countries, on='user_id')
df_merged

,session_id,ip_address,devices,OS,browser,date_login,date_logout,user_id,active_month,professional_id,gender,english_level,looking_job,location_preference,location,active,user_created_at,role_id,created_month,country_id,country_name
0,1502,116.122.16.192,smartphone,"""OS"" X",Opera,2021-03-16 18:55:14,2021-03-16 19:37:40,573,3.0,573,Cis,C2,True,Brazil,2,True,2021-05-23 21:02:22,1,5.0,2,Bolivia
1,2475,243.44.168.190,smartphone,"""OS"" X",Unknown,2021-09-03 07:00:15,2021-09-03 09:04:20,573,9.0,573,Cis,C2,True,Brazil,2,True,2021-05-23 21:02:22,1,5.0,2,Bolivia
2,1501,22.163.32.101,desktop,"Chrome ""OS""",Chrome,2021-06-05 04:39:48,2021-06-05 06:39:54,542,6.0,542,Cis Female,B1,False,Costa Rica,13,True,2021-07-27 20:10:48,2,7.0,13,Peru
3,2109,127.173.105.47,TV,Unknown,Edge,2021-10-05 05:34:08,2021-10-05 06:32:37,542,10.0,542,Cis Female,B1,False,Costa Rica,13,True,2021-07-27 20:10:48,2,7.0,13,Peru
4,1503,119.228.238.82,TV,Unknown,Chrome,2021-05-07 12:17:19,2021-05-07 13:27:47,642,5.0,642,Androgynous,A1,True,Mexico,3,True,2021-07-22 15:41:09,2,7.0,3,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2453,112.50.69.202,desktop,"Chrome ""OS""",Chrome,2021-03-23 23:17:38,2021-03-24 01:36:57,300,3.0,300,Transsexual Man,A2,False,Colombia,4,False,2021-10-01 23:13:12,2,10.0,4,Chile
996,2462,211.81.150.123,smartphone,Others,Unknown,2021-12-15 17:52:49,2021-12-15 20:15:17,68,12.0,68,Agender,B2,False,Colombia,4,True,2021-01-24 21:27:17,2,1.0,4,Chile
997,2471,87.73.63.103,smartphone,Others,Unknown,2021-11-29 17:07:18,2021-11-29 17:26:11,520,11.0,520,Genderqueer,A1,True,Brazil,3,False,2021-07-29 03:49:49,2,7.0,3,Brazil
998,2490,209.56.38.146,desktop,"Chrome ""OS""",Chrome,2021-09-30 18:20:34,2021-09-30 20:08:27,608,9.0,608,Male to Female,B1,False,Brazil,9,False,2021-01-24 20:22:05,2,1.0,9,Honduras


# 6. Open a spreadsheet to receive the cleaned data

In [ ]:
json_path = '/content/drive/MyDrive/Dashboard/dashboard_key.json'

In [ ]:
g_sheets = pygsheets.authorize(service_file = json_path)

In [ ]:
data_sheet = g_sheets.open('postgres_data')
postgres_data = data_sheet.sheet1


# 7. Send data to Google sheet

In [ ]:
postgres_data.set_dataframe(df_merged,(1,1))